In [1]:
import os
import json
import math
import price
import ratios
import requests
import momentum
import pyticker
import dividend
import questrade
import auto_email
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import pandas_datareader.data as web

## Global Variables

In [2]:
WATCHLIST = ['O']
MOMENTUM_PERIODS = [3,6,12,24,36,48,60]
DIV_GROWTH_RATE_THRESHOLD = 0.1
MOMENTUM_THRESHOLD = 0.50

## Create S&P 500 Data If Not Exists

In [3]:
if not os.path.exists('./sp500_100_billions_symbols.json'):
    sp500 = pyticker.get_symbols_by_index('S&P 500')

    data = {'Symbol': [], 'Market_Cap (B)': []}

    market_cap_threshold = 200

    count = 0

    for symbol in sp500:
        count += 1
        print(f"{symbol}: {count}/{len(sp500)}")
        try:
            market_cap = ratios.calculate_market_cap(symbol)
        except:
            continue

        if market_cap >= market_cap_threshold and dividend.exists_dividends(symbol):
            data['Symbol'].append(symbol)
            data['Market_Cap (B)'].append(market_cap)
            print(f"{symbol} has been added!")

    with open('./sp500_100_billions_symbols.json', 'w') as fp:
        json.dump(data,fp)

## Load Data

In [4]:
with open('./sp500_100_billions_symbols.json', 'r') as fp:
    data = json.load(fp)
df = pd.DataFrame(data)
df.set_index('Symbol', inplace=True)

## Update Watchlist

In [5]:
watchlist_data = {'Symbol': [], 'Market_Cap (B)': []}
for symbol in WATCHLIST:
    if len(WATCHLIST) == 1:
        watchlist_data['Symbol'] = symbol
        watchlist_data['Market_Cap (B)'] = ratios.calculate_market_cap(symbol)
    else:
        watchlist_data['Symbol'].append(symbol)
        watchlist_data['Market_Cap (B)'].append(ratios.calculate_market_cap(symbol))

df.reset_index(inplace=True)
df = df.append(watchlist_data, ignore_index=True)
df.set_index('Symbol', inplace=True)


## Dividend Growth and Momentum

In [6]:
for symbol in list(df.index):
    try:
        div_growth = dividend.calcualte_avg_dividend_growth(symbol,10)
        if div_growth == float('inf'):
            df.loc[symbol, 'Dividend_Growth'] = np.nan
        else:
            df.loc[symbol, 'Dividend_Growth'] = div_growth
    except:
        df.loc[symbol, 'Dividend_Growth'] = np.nan
        
    try:
        mom = momentum.calculate_equal_weight_momentum(symbol, MOMENTUM_PERIODS)
        df.loc[symbol,'Momentum'] = mom
    except:
        df.loc[symbol,'Momentum'] = np.nan

df.dropna(inplace=True)

In [7]:
df = df[(df['Dividend_Growth'] > DIV_GROWTH_RATE_THRESHOLD) & (df['Momentum'] > MOMENTUM_THRESHOLD)]
df

,Market_Cap (B),Dividend_Growth,Momentum
Symbol,,,
AAPL,2257.159507,0.238712,1.565962
HD,346.899390,0.206233,0.759093
JPM,457.758062,0.468526,0.798693
MSFT,1907.124687,0.144052,1.413311
NKE,208.476659,0.137093,0.724494
UNH,372.739439,0.284789,0.813156
V,503.413918,0.253689,0.691086
DIS,335.025105,0.180461,0.665096
NVDA,383.077106,0.559168,3.223258


## Export DataFrame

In [8]:
df.to_csv(r'./export_df_rpi.csv')

## Drawdowns from 52W High

In [9]:
for symbol in list(df.index):
    high = price.calculate_prev_max_high(symbol,252)
    curr_price = price.get_current_price(symbol)
    df.loc[symbol,'12M_High'] = high
    df.loc[symbol,'Current_Price'] = curr_price
    df.loc[symbol,'15%_Drop'] = high * 0.85
    df.loc[symbol,'30%_Drop'] = high * 0.70
    df.loc[symbol,'50%_Drop'] = high * 0.5    
df

,Market_Cap (B),Dividend_Growth,Momentum,12M_High,Current_Price,15%_Drop,30%_Drop,50%_Drop
Symbol,,,,,,,,
AAPL,2257.159507,0.238712,1.565962,145.089996,134.389999,123.326497,101.562997,72.544998
HD,346.899390,0.206233,0.759093,328.829987,320.589996,279.505489,230.180991,164.414993
JPM,457.758062,0.468526,0.798693,161.690002,151.250000,137.436502,113.183002,80.845001
MSFT,1907.124687,0.144052,1.413311,262.440002,261.970001,223.074002,183.708002,131.220001
NKE,208.476659,0.137093,0.724494,147.949997,132.110001,125.757497,103.564998,73.974998
UNH,372.739439,0.284789,0.813156,401.989990,394.500000,341.691492,281.392993,200.994995
V,503.413918,0.253689,0.691086,232.949997,229.910004,198.007497,163.064998,116.474998
DIS,335.025105,0.180461,0.665096,203.020004,184.639999,172.567004,142.114003,101.510002
NVDA,383.077106,0.559168,3.223258,648.570007,615.270020,551.284506,453.999005,324.285004
